In [1]:
import pandas as pd
import geopandas as gp
from shapely.geometry import Point
import matplotlib as plt
import numpy as np
%matplotlib inline

In [2]:
df = pd.read_csv("tx_stops_2010_2015.csv", encoding="ISO-8859-1")

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (17,20,25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.columns

Index(['Unnamed: 0', 'id', 'state', 'stop_date', 'stop_time', 'location_raw',
       'county_name', 'county_fips', 'fine_grained_location',
       'police_department', 'driver_gender', 'driver_age_raw', 'driver_age',
       'driver_race_raw', 'driver_race', 'violation_raw', 'violation',
       'search_conducted', 'search_type_raw', 'search_type',
       'contraband_found', 'stop_outcome', 'is_arrested', 'lat', 'lon',
       'officer_id', 'driver_race_original'],
      dtype='object')

In [4]:
df = df[df.county_name == 'Harris County']

## Baseline Stats: Harris County

In [5]:
print(100*df.search_conducted.dropna().astype(int).mean(), 100*df.search_conducted.dropna().astype(int).std())

1.26272103934 11.1659343294


In [6]:
print(100*df.contraband_found.dropna().astype(int).mean(), 100*df.contraband_found.dropna().astype(int).std())

0.462650532842 6.78610152879


In [7]:
100*len(df[df.stop_outcome=='Citation'])/len(df)

48.689323716294965

In [8]:
100*len(df[df.driver_gender=='M'])/len(df)

64.9731009175636

In [9]:
100*len(df[df.driver_race=='White'])/len(df)

36.45357878670886

In [10]:
100*len(df[df.driver_race=='Black'])/len(df)

24.382006660725175

In [11]:
100*len(df[df.driver_race=='Hispanic'])/len(df)

30.82587940695732

In [12]:
100*len(df[df.driver_race=='Asian'])/len(df)

4.878277592757634

In [13]:
100*len(df[df.driver_race=='Other'])/len(df)

0.2500478085636864

In [14]:
100*len(df[df.driver_race.isnull()])/len(df)

3.2102097442873276

In [15]:
## Stats on Searches and Contraband Recovered, % Citations by Race

In [16]:
df1 = df[df.driver_race == 'White']
print(100*df1.search_conducted.dropna().astype(int).mean(), 100*df1.search_conducted.dropna().astype(int).std())
print(100*df1.contraband_found.dropna().astype(int).mean(), 100*df1.contraband_found.dropna().astype(int).std())
print(100*len(df1[df1.stop_outcome=='Citation'])/len(df1))

1.04439934663 10.1661294397
0.388061179033 6.2173876427
43.30198950806691


In [17]:
df2 = df[df.driver_race == 'Black']
print(100*df2.search_conducted.dropna().astype(int).mean(), 100*df2.search_conducted.dropna().astype(int).std())
print(100*df2.contraband_found.dropna().astype(int).mean(), 100*df2.contraband_found.dropna().astype(int).std())
print(100*len(df2[df2.stop_outcome=='Citation'])/len(df2))

1.81468324452 13.3483289779
0.648312611012 8.02570278933
46.80202444727262


In [18]:
df3 = df[df.driver_race == 'Hispanic']
print(100*df3.search_conducted.dropna().astype(int).mean(), 100*df3.search_conducted.dropna().astype(int).std())
print(100*df3.contraband_found.dropna().astype(int).mean(), 100*df3.contraband_found.dropna().astype(int).std())
print(100*len(df3[df3.stop_outcome=='Citation'])/len(df))

1.2210398155 10.9824612611
0.443700391018 6.64632877701
17.750147033884545


Statesman Methodology

To compare search rates by race, we looked at individual troopers who had at least 100 stops and 20 searches of either black or Hispanic motorists, and then compared the number of searches to stops to get a search rate for that race. We did the same for their stops of white motorists, and then divided the rates to get a ratio of minority stops. For contraband hit rates, we compared the number of positive contraband results to the total number of searches by race for each trooper. DPS misidentified many Hispanic motorists as Anglo during the period studied, meaning the paper’s conclusions are most likely conservative.

35 percent of the 1,138 troopers included in the analysis searched black and Hispanic motorists at least twice as often as white drivers.

231 of the officers who searched black and Hispanic motorists at two times or more the rate at which they searched white drivers were less likely to find contraband while searching the minority drivers.

65 DPS officers searched minority drivers at least three times more often than the white motorists they stopped yet found contraband less often.

16 officers searched minority motorists more than four times as often as Anglos, with lower contraband hit rates.

In [19]:
df.columns

Index(['Unnamed: 0', 'id', 'state', 'stop_date', 'stop_time', 'location_raw',
       'county_name', 'county_fips', 'fine_grained_location',
       'police_department', 'driver_gender', 'driver_age_raw', 'driver_age',
       'driver_race_raw', 'driver_race', 'violation_raw', 'violation',
       'search_conducted', 'search_type_raw', 'search_type',
       'contraband_found', 'stop_outcome', 'is_arrested', 'lat', 'lon',
       'officer_id', 'driver_race_original'],
      dtype='object')

In [20]:
df_filt = df.groupby("officer_id").count()
df_filt['id'] = df_filt.index
df_filt.head()
ids = np.asarray(df_filt.id[df_filt.state >= 100])

In [21]:
df_filt = df[(df.driver_race == 'Hispanic') | (df.driver_race == 'Black')].groupby("officer_id").count()
df_filt['id'] = df_filt.index
df_filt.head()
ids1 = np.asarray(df_filt.id[df_filt.state >= 20])

In [22]:
ids_filt = np.intersect1d(ids.astype(str), ids1.astype(str))

In [25]:
test.index.astype(str)

Index(['127', '154', '200', '213', '227', '228', '343', '382', '392', '399',
       ...
       '11130958', '11130972', '11130980', '11130989', '11130991', '11131036',
       '11131039', '11131063', '11131091', '11131099'],
      dtype='object', length=277147)